In [83]:
# from google.colab import drive
# drive.mount('/content/gdrive')
!unzip 'speech_data.zip'

Archive:  speech_data.zip
replace speech_data/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: speech_data/.DS_Store   
replace speech_data/commands/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: speech_data/commands/.DS_Store  
replace speech_data/commands/test/down/down_1.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: speech_data/commands/test/down/down_1.wav  
replace speech_data/commands/test/down/down_2.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: speech_data/commands/test/down/down_2.wav  
replace speech_data/commands/test/down/down_3.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: speech_data/commands/test/down/down_3.wav  
replace speech_data/commands/test/down/down_4.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: speech_data/commands/test/down/down_4.wav  
replace speech_data/commands/test/down/down_5.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: speech_data/commands/test/down/down_5.wav  

In [0]:
commands_train_dir='speech_data/commands/train/'
commands_test_dir='speech_data/commands/test/'

digits_train_dir='speech_data/digits/train/'
digits_test_dir='speech_data/digits/test/'

In [0]:
import numpy as np
import os
import tensorflow as tf

In [0]:
def load_graph(filename,name=''):
  """Unpersists graph from file as default graph."""
  with tf.gfile.FastGFile(filename, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name=name)
  return graph_def

def run_graph(sess,output_tensor,wav_file):
  with open(wav_file, 'rb') as wav_file:
      wav_data = wav_file.read()
  input_layer_name = 'wav_data:0'
  predictions = sess.run(output_tensor, {input_layer_name: wav_data})
  return predictions        
    
def is_wav(path):
  _, file_extension = os.path.splitext(path)
  return file_extension.lower() in ['.wav']

In [185]:
tf.reset_default_graph()
graph_def=load_graph('speech_data/conv_actions_frozen.pb')
print([n.name for n in graph_def.node])

['wav_data', 'decoded_sample_data', 'AudioSpectrogram', 'Mfcc', 'Reshape/shape', 'Reshape', 'Variable', 'Variable/read', 'Variable_1', 'Variable_1/read', 'Conv2D', 'add', 'Relu', 'MaxPool', 'Variable_2', 'Variable_2/read', 'Variable_3', 'Variable_3/read', 'Conv2D_1', 'add_1', 'Relu_1', 'Reshape_1/shape', 'Reshape_1', 'Variable_4', 'Variable_4/read', 'Variable_5', 'Variable_5/read', 'MatMul', 'add_2', 'labels_softmax']


In [0]:
command_labels=['yes','no','up','down','left','right','on','off','stop','go']


In [0]:

def get_features_lables(commands_dir, layer):
  with tf.Session() as sess:
    features = []
    labels = []
    out_tensor = sess.graph.get_tensor_by_name(layer)
    acc=0
    num_tests=0
    for d in next(os.walk(commands_dir))[1]:
      for f in next(os.walk(os.path.join(commands_dir,d)))[2]:
        if is_wav(f):
          wav_file=os.path.join(commands_dir,d,f)        
          preds=run_graph(sess,out_tensor,wav_file)[0]
          
          
          features.append(preds)
          labels.append(command_labels.index(d))
          
    return features, labels
  

In [188]:
train_features, train_labels = get_features_lables(commands_train_dir, "Reshape_1:0")
test_features, test_labels = get_features_lables(commands_test_dir, "Reshape_1:0")


np.array(train_features).shape


(100, 64000)

In [189]:
# SVC

from sklearn.svm import SVC
svc = SVC()


svc.fit(train_features, train_labels)

err_train = np.mean(train_labels != svc.predict(train_features))
err_test  = np.mean(test_labels  != svc.predict(test_features))

print(err_train, err_test)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.29 0.6326530612244898


In [0]:
train_features, train_labels = get_features_lables(commands_train_dir, "MatMul:0")
test_features, test_labels = get_features_lables(commands_test_dir, "MatMul:0")

In [164]:
# SVC

from sklearn.svm import SVC
svc = SVC()


svc.fit(train_features, train_labels)

err_train = np.mean(train_labels != svc.predict(train_features))
err_test  = np.mean(test_labels  != svc.predict(test_features))

print(err_train, err_test)

0.07 0.1836734693877551


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [0]:
train_features, train_labels = get_features_lables(digits_train_dir, "Reshape_1:0")
test_features, test_labels = get_features_lables(digits_test_dir, "Reshape_1:0")

In [166]:
# SVC

from sklearn.svm import SVC
svc = SVC()


svc.fit(train_features, train_labels)

err_train = np.mean(train_labels != svc.predict(train_features))
err_test  = np.mean(test_labels  != svc.predict(test_features))

print(err_train, err_test)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.2923076923076923 0.4752475247524752


In [0]:
#Check out other layers for feature extraction. Start from Reshape_1 instead of labels_softmax